In [17]:
import pandas as pd
import pyodbc
import sqlalchemy as sa
import psycopg2 as p
import os
import array
import string
import json
import csv

In [18]:
# This script reads all play-by-play flattened json files stored in the same folder, 
# normalizes them, and adds them to a postgres database. 

# path to folder 
filePath = r'C:\Users\mmgri\Desktop\flattened'

# creating database connection string
conn = p.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=mmgtest user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()

# list for dataframes
dfs = []

# iterate through files in folder
for file in os.listdir(filePath):
    fileName = os.fsdecode(file)
    
    # find all json files
    if fileName.endswith(".json"): 
        
        # read json file to dataframe
        df = pd.read_json(os.path.join(filePath, fileName), orient='columns')
        
        # add dataframe to list of dataframes
        dfs.append(df)
        continue
    else:
        continue

# combine list into one dataframe
df = pd.concat(dfs)

# remove commas from descriptions to avoid erros
df['description'] = df['description'].str.replace('[^\w\s]', '')

# normalize dataframes 
dfGame = df[['awayAbbr', 'awayId', 'awayTeam', 'gameId', 'homeAbbr', 'homeId', 'homeTeam', 'week', 'year']]
dfPlay = df[['awayScore', 'clock', 'defenseAbbr', 'defenseId', 'defenseTeam', 'description', 'distance', 'down', 
             'driveIndex', 'endYardLine', 'gameId', 'homeScore', 'offenseAbbr', 'offenseId', 
             'offenseTeam', 'playIndex', 'quarter', 'type', 'yardLine', 'yardsGained']]

# remove duplicates from game
dfGame.drop_duplicates(subset ="gameId", keep = 'first', inplace = True) 

# array for file names and table names 
fileNames = ['game', 'play']

# array for dormalized dataframes
dfNormalized = [dfGame, dfPlay]

# loop to add files to postgres database
for df, file in zip(dfNormalized, fileNames):
    print(df.shape)
    print(file)
    
    # writes dataframe to a csv 
    df.to_csv(r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv', sep=',', index=False)
    fileString = r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv'
    
    # loads csv to database 
    with open(fileString, 'r') as f:
        next(f)
        cur = conn.cursor()
        cur.copy_from(f, file, sep=',')
        conn.commit()


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(48, 9)
game
(8749, 20)
play
